<a href="https://colab.research.google.com/github/sugarforever/LangChainSummarizeYoutubeTranscript/blob/main/LangChainSummarizeYoutubeTranscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install youtube_transcript_api
%pip install openai
%pip install langchain
%pip install unstructured
%pip install tiktoken

In [ ]:
%pip install langchain -U

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

In [ ]:
transcript_list = YouTubeTranscriptApi.list_transcripts('UF9Iqmg94tk')
print(transcript_list)

In [ ]:
# trz7g-wilxs = "5 RULES FOR THE REST OF YOUR LIFE" | Matthew McConaughey MOTIVATIONAL SPEECH
# UF9Iqmg94tk = "Consistent Hashing | Algorithms You Should Know #1"
transcript = YouTubeTranscriptApi.get_transcript('UF9Iqmg94tk', languages=['en-US'])
formatter = TextFormatter()
text_formatted = formatter.format_transcript(transcript)
with open('transcript.txt', 'w', encoding='utf-8') as text_file:
    text_file.write(text_formatted)

In [ ]:
%cat transcript.txt

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader("./transcript.txt")
docs = loader.load()

In [ ]:
print (f'You have {len(docs)} document(s) in your data')
print (f'There are {len(docs[0].page_content)} characters in your document')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print (f'You have {len(split_docs)} split document(s)')

In [ ]:
import os
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name="text-davinci-002")
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)

In [ ]:
input_docs = split_docs[:2]

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    chain.run(input_documents=input_docs)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")